In [5]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [6]:
cwd = os.getcwd()

In [17]:
df_raw = pd.read_csv(cwd+'/data.csv', index_col=0)

/tmp/ipykernel_110286/4123645007.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(cwd+'/data.csv', index_col=0)


In [19]:
df = df_raw.copy()

In [20]:
df.isnull().sum()

itemid                           0
product_name                     0
brand                        40522
price_min                        0
price                            0
price_max                        0
price_min_before_discount        0
price_before_discount            0
price_max_before_discount        0
historical_sold                  0
sold                             0
category0                        0
category1                        0
category2                     2113
category3                    17062
category4                    68917
fe_category0                     0
fe_category1                     0
fe_category2                    17
normal_stock                     0
discount_stock                   0
other_stock                      0
raw_discount                     0
ctime                            0
liked_count                      0
cmt_count                        0
shop_name                        0
shop_location                   14
rating_star         